In [ ]:
# import own 共用 lib code
import sys
sys.path.append(r"../data/data_checking_code/")
sys.path.append(r"./3_import_function/")

import post_proc
import folder_tool
import tool
import vis_tool

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

import cv2
import tensorflow.keras as keras
import shutil
import numpy as np
import matplotlib.pyplot as plt
import glob

from tqdm.notebook import tqdm

In [ ]:
# check output folder
for img_type in ['train', 'val', 'test']:
    folder_tool.check_folder('./result_post_process/{}_pred_raw'.format(img_type))
    folder_tool.check_folder('./result_post_process_view/{}_pred_raw'.format(img_type))
folder_tool.check_folder('./result_post_process/seg_raw/')

In [ ]:
# Input
# TODO 使用 CEJ 圖片
CEJ_images_folder = '' # run all images in ./result/
# CEJ_images_folder = './result/seg_raw/'
if CEJ_images_folder:
    CEJ_img_paths = glob.glob(CEJ_images_folder + '/*.PNG')
else:
    # CEJ_img_paths = glob.glob('./record/512_epoch_80_on42/result/*_raw/*.PNG')
    CEJ_img_paths = glob.glob('./result/*_raw/*.PNG')
# TODO input 會 resize 成 img_size*img_size 的圖片
# img_size = 512
img_size = 1024
black_img = np.zeros((img_size, img_size))
algro = {
    'a_thresh': -1,
    # 'a_thresh': img_size*0.25,
    # -1 means close
    'b_thresh': 1024,
    # 1024 means close
    'c_process': True,
    'link_edge': True
}
DEBUG = False

In [ ]:
tar_img_id = [
    "NN_180607_095256_99242E",
    # "NN_180316_154203_C02AE2",
    # "NN_120823_103715_4AA86"
    # 'NN_120827_095346_BFFF36' problem with cv2 filled
    # "NN_100825_103844_C0381C",
    # "NN_100825_103850_C0381F",
    # "NN_120827_095402_BFFF3E",
    # "NN_170619_152245_16F2F5",
    # "NN_171005_134750_98B6EE",
    # "NN_180122_144804_16F0BD",
    # "NN_180717_140845_98BC97",
    # "NN_180717_140916_98BC9B"
]

In [ ]:
for img_path in tqdm(CEJ_img_paths):
    # get cej point
    if len(tar_img_id) != 0 and \
            os.path.basename(img_path)[:-4] not in tar_img_id:
        continue
    # print(os.path.basename(img_path))

    CEJ_mask = cv2.imread(img_path, 0)
    CEJ_mask = cv2.resize(CEJ_mask, (img_size, img_size), interpolation=cv2.INTER_NEAREST)
    if len(np.unique(CEJ_mask)) != 2:
        print(img_path)
    assert len(np.unique(CEJ_mask)) <= 2, np.unique(CEJ_mask)
    pixel_y_cord_list = np.where(CEJ_mask == 255)[0]
    # vis_tool.dist_vis(pixel_y_cord_list, 5, os.path.basename(img_path), 20)


    contours, _ = cv2.findContours(CEJ_mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = lambda x: np.mean(np.transpose(x)[0]))
    CEJ_link_mask = black_img.copy()
    vis_process_mask = cv2.cvtColor(black_img.copy().astype(np.uint8), cv2.COLOR_GRAY2BGR)
    choose, not_choose = black_img.copy(), black_img.copy()
    if algro['c_process']:
        weighted_mask = black_img.copy()
        index_dict = {}
        cont_size = []
        for id, cont in enumerate(contours):
            cont_size.append((id, cv2.contourArea(cont)))
        cont_size = sorted(cont_size, key = lambda x: x[1])
        for rank, cont in enumerate(cont_size):
            index_dict[cont[0]] = rank + 1
        for id, cont in enumerate(contours):
            cv2.drawContours(weighted_mask, [cont], -1, index_dict[id], cv2.FILLED)
        if DEBUG:
            vis_tool.visualize([weighted_mask])
        # weighted_mask = cv2.applyColorMap((255 - weighted_mask).astype(np.uint8), cv2.COLORMAP_JET)
        # vis_tool.visualize([weighted_mask])
        for col_id in range(img_size):
            is_max = weighted_mask[:, col_id] == max(weighted_mask[:, col_id])
            choose[:, col_id] = weighted_mask[:, col_id] * is_max
            not_choose[:, col_id] = weighted_mask[:, col_id] * (~ is_max)
        choose = np.where(choose != 0, 1, 0)
        not_choose = np.where(not_choose != 0, 1, 0)
        CEJ_mask = CEJ_mask * choose

        contours, _ = cv2.findContours(CEJ_mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key = lambda x: np.mean(np.transpose(x)[0]))

    if algro['a_thresh'] != -1:
        ori_contours = contours
        contours = post_proc.drop_outlyers_cont(contours, CEJ_mask, vis_process_mask, img_size, algro['a_thresh'])

    ex_cont_pt = []
    process_b_drop_cont_mask = black_img.copy()
    for cont in contours:
        start_pt = min(cont, key = lambda x: (x[0][0], x[0][1]))[0]
        end_pt = max(cont, key = lambda x: (x[0][0], x[0][1]))[0]
        # left most point
        if len(ex_cont_pt) > 0:
            if abs(start_pt[1] - ex_cont_pt[1]) >= algro['b_thresh']:
                cv2.drawContours(process_b_drop_cont_mask, [cont], -1, 255, cv2.FILLED)
                continue
            else:
                cv2.line(CEJ_link_mask, ex_cont_pt, start_pt, 255, 1)
        elif algro['link_edge']:
            # link the edge
            cv2.line(CEJ_link_mask, (0, start_pt[1]), start_pt, 255, 1)
        cv2.drawContours(CEJ_link_mask, [cont], -1, 255, cv2.FILLED)
        ex_cont_pt = end_pt
    if len(ex_cont_pt) != 0 and algro['link_edge']:
        cv2.line(CEJ_link_mask, ex_cont_pt,(img_size-1, ex_cont_pt[1]), 255, 1)
    o_img = cv2.imread('../data/1_o_image_512/' + os.path.basename(img_path), 0)
    vis_process_mask = vis_tool.draw_binary_on_image(vis_process_mask, CEJ_link_mask, (255, 255, 255))
    vis_process_mask = vis_tool.draw_binary_on_image(vis_process_mask, process_b_drop_cont_mask, (255, 0, 255))
    vis_process_mask = vis_tool.draw_binary_on_image(vis_process_mask, not_choose, (0, 255, 0))
    # if len(contours) != len(ori_contours):
    if DEBUG:
        vis_tool.visualize([CEJ_mask, CEJ_link_mask, vis_process_mask, o_img], width=4)
        # pass
    # vis_tool.visualize([CEJ_mask, CEJ_link_mask, vis_process_mask, o_img], width=4)

    vis_tar_path = img_path.replace('result', 'result_post_process_view')
    cv2.imwrite(vis_tar_path, vis_process_mask)
    target_path = img_path.replace('result', 'result_post_process')
    cv2.imwrite(target_path, CEJ_link_mask)
    # CEJ_link_test = cv2.imread(target_path, 0)
    # assert len(CEJ_link_test.shape) == 2, CEJ_link_test.shape
    # assert (np.unique(CEJ_link_test) == np.unique(CEJ_mask)).all(), np.unique(CEJ_link_test)
